In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from lib import Datasets

In [2]:
# model_checkpoint = "distilgpt2"
model_name = "roberta-base"
device = 'cuda:0'

# model_checkpoint = "NousResearch/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, is_decoder=True).to(device)

lm_datasets = Datasets.LoadWiki(tokenizer, 256, 128)
# lm_datasets = Datasets.LoadWiki(tokenizer, 512, 128)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [4]:
from transformers import Trainer, TrainingArguments

name = model_name.split("/")[-1]

In [6]:
import torch

X = """ The game takes place during the Second Europan War . Gallian Army Squad 422 , also known as " The Nameless " , are a penal military unit composed of criminals , foreign deserters , and military offenders whose real names are erased from the records and thereon officially referred to by numbers . Ordered by the Gallian military to perform the most dangerous missions that the Regular Army and Militia will not do , they are nevertheless up to the task , exemplified by their motto , Altaha Abilia , meaning " Always Ready . " The three main characters are No.7 Kurt Irving , an army officer falsely accused of treason who wishes to redeem himself ; Ace No.1 Imca , a female Darcsen heavy weapons specialist who seeks revenge against the Valkyria who destroyed her home ; and No.13 Riela Marcellis , a seemingly jinxed young woman who is unknowingly a descendant of the Valkyria . Together with their fellow squad members , these three are tasked to fight against a mysterious Imperial unit known as Calamity Raven , consisting of mostly Darcsen soldiers . """

inputs = tokenizer(X, return_tensors='pt', truncation=True).to(device)

inputs['labels'] = inputs['input_ids'].clone()

input_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)

print(input_ids, attention_mask)

tensor([[    0,    20,   177,  1239,   317,   148,     5,  4665,  5122, 12560,
          1771,   479,  7155,   811,  2938, 19200, 41977,  2156,    67,   684,
            25,    22,    20,  8603, 13802,    22,  2156,    32,    10, 14914,
           831,  1933, 14092,     9,  9177,  2156,  1093,  2694,   254,  2696,
          2156,     8,   831, 11553,  1060,   588,  2523,    32, 23717,    31,
             5,  2189,     8,    89,   261,  4142,  4997,     7,    30,  1530,
           479, 16702,  3215,    30,     5,  7155,   811,   831,     7,  3008,
             5,   144,  2702, 10779,    14,     5, 24306,  2938,     8, 47059,
            40,    45,   109,  2156,    51,    32, 21507,    62,     7,     5,
          3685,  2156, 22249,  3786,    30,    49, 24609,  2156,  7330, 11695,
          2060, 10990,  2156,  3099,    22, 14947, 19325,   479,    22,    20,
           130,  1049,  3768,    32,   440,     4,   406, 12255, 11241,  2156,
            41,  3835,  1036, 19349,  1238,     9, 2

In [7]:
from lib import Compressors, Globals

names = ['attention.self.query', 'attention.self.key', 'attention.self.value']
# ranks = [256, 256, 256]
ranks = [768, 768, 768]
# p = Globals.Positions(model, 'roberta.encoder.layer', names, ranks)

# Compressors.CompressSVD(p, 12)

# Compressors.CompressSpecial(p, input_ids, attention_mask, 12, Compressors.AuthorFindWeightNorm, inputs)


# Compressors.CompressSpecial(p, input_ids, attention_mask, 12, Compressors.JustSVD, trainer0, inputs)

# Compressors.CompressSpecial(p, input_ids, attention_mask, 12, Compressors.MyFindWeightNormOLD)
# Compressors.CompressSpecial(p, input_ids, attention_mask, 12, Compressors.AuthorFindWeightNorm)


In [8]:
training_args = TrainingArguments(
    f"./workout/{name}-test",
    eval_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=10,
    logging_dir=f"./logs/{name}-test",
    push_to_hub=False,
    # save_strategy="epoch",
    # use_cpu=True,
    # bf16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [ ]:
# trainer.train()

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.4f}")

/home/parshina/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Perplexity: 3446.8498
